<h3 align="center"> Movie Recommendation Sytem <hr></h3>
<br>

## Import the libraries

In [1]:
import pandas as pd 
import numpy as np 

import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# loading the data from the csv file to apandas dataframe

movies = pd.read_csv("movies.csv")

In [3]:
# The first five rows of the dataset

movies.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [4]:
# All the columns of tha dataframe 

movies.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

In [5]:
# The dimensions of the dataset 

movies.shape

(4803, 24)

In [6]:
# Checking for missing values and NaN values 
movies.isnull().sum()

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

In [7]:
# Select the most relevant features for the upcoming recommendation system 

selected_features = ['genres','keywords','tagline','cast','director']

In [8]:
# Create a new dataframe of the most important features 

movies[selected_features].head(3)

,genres,keywords,tagline,cast,director
0,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,Enter the World of Pandora.,Sam Worthington Zoe Saldana Sigourney Weaver S...,James Cameron
1,Adventure Fantasy Action,ocean drug abuse exotic island east india trad...,"At the end of the world, the adventure begins.",Johnny Depp Orlando Bloom Keira Knightley Stel...,Gore Verbinski
2,Action Adventure Crime,spy based on novel secret agent sequel mi6,A Plan No One Escapes,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,Sam Mendes


In [9]:
# Replace the missing rows with 0 

for feature in selected_features:
    movies[feature] = movies[feature].fillna('')

In [10]:
# Combine all the five selected fetaures 

def get_features(data):
    combined_features = []
    for i in range (0,movies.shape[0]):
        combined_features.append(movies['genres'][i] + ' ' + 
                                 movies['keywords'][i] + ' ' + movies['tagline'][i] +
                                 ' ' + movies['cast'][i] + ' ' + movies['director'][i])
        
    return combined_features

In [11]:
movies['important_features'] = get_features(movies)

In [12]:
movies['important_features']

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Name: important_features, Length: 4803, dtype: object

In [13]:
# The new dataset 

movies.head(2)

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director,important_features
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron,Action Adventure Fantasy Science Fiction cultu...
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski,Adventure Fantasy Action ocean drug abuse exot...


* Which is better CountVectorizer or TfidfVectorizer?

    TF-IDF is better than Count Vectorizers because it not only focuses on the frequency of words present     in the corpus but also provides the importance of the words. 
    
    We can then remove the words that are less important for analysis, hence making the model building       less complex by reducing the input dimensions.

In [14]:
# Converting the text data to feature vectors

vectorizer = TfidfVectorizer()

In [15]:
feature_vectors = vectorizer.fit_transform(movies['important_features'])

In [16]:
print(movies['important_features'])

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Name: important_features, Length: 4803, dtype: object


* Cosine Similarity

In [17]:
# getting the similarity scores using cosine similarity

similarity = cosine_similarity(feature_vectors)

In [18]:
similarity

array([[1.        , 0.07219487, 0.037733  , ..., 0.        , 0.        ,
        0.        ],
       [0.07219487, 1.        , 0.03281499, ..., 0.03575545, 0.        ,
        0.        ],
       [0.037733  , 0.03281499, 1.        , ..., 0.        , 0.05389661,
        0.        ],
       ...,
       [0.        , 0.03575545, 0.        , ..., 1.        , 0.        ,
        0.02651502],
       [0.        , 0.        , 0.05389661, ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.02651502, 0.        ,
        1.        ]])

In [19]:
# Get the shape of the cosine similarity matrix  
# Basicaly, we will view a matrix which will have as columns and rows 
# the number of rows of the original dataset 

similarity.shape

(4803, 4803)

Get the movie name as input from a user 

In [20]:
# getting the movie name from the user

movie_name = input(' Enter your favourite movie name : ')

 Enter your favourite movie name :  Fight Club


In [21]:
# creating a list with all the movie names given in the dataset

list_of_all_titles = movies['title'].tolist()

In [22]:
# finding the close match for the movie name given by the user

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)

['Fight Club', 'Fight Valley', 'Flight']


In [23]:
# Find the closest movie 

close_match = find_close_match[0]
print(close_match)

Fight Club


In [24]:
# Find the index of the movie with the title

index_movie = movies[movies.title == close_match]['index'].values[0]

In [25]:
# Gather all the similar movies in a list 

similar_movies = list(enumerate(similarity[index_movie]))

* Create a list of enumerations for the similar movies 

    The output should look like this 

    [ ( index_movie, similarity ]

In [26]:
similar_movies[:10]

[(0, 0.0),
 (1, 0.0),
 (2, 0.0),
 (3, 0.03172085537943224),
 (4, 0.0),
 (5, 0.07556019661945429),
 (6, 0.0),
 (7, 0.0),
 (8, 0.011629977907314024),
 (9, 0.0)]

In [27]:
# The lenght of the similar movies 

len(similar_movies)

4803

In [28]:
# Sort the movies by the similarity of them that's why we use 1 

sort_similar_movies = sorted(similar_movies, key = lambda x:x[1], reverse = True)

In [29]:
# We don't want the first movie a user types, but we want us to find similars 

sort_similar_movies = sort_similar_movies[1:]

So, here we see the movies id and the similarity the have 

In [30]:
sort_similar_movies[:10]

[(2634, 0.18054537801897236),
 (1010, 0.16231237698291315),
 (3250, 0.1588244855154259),
 (3393, 0.15510505687415804),
 (3894, 0.14184191462427292),
 (1509, 0.12862057749571845),
 (326, 0.1193013342681584),
 (105, 0.11733158650222146),
 (1771, 0.11729752354551987),
 (3270, 0.11688618414270738)]

In [31]:
# Print the first ten  movies based on their similarity 

print(f'The most recommended movies to  {movie_name} are: \n')

i = 0

for movie in sort_similar_movies:
    
    index = movie[0]
    
    title_from_index = movies[movies.index==index]['title'].values[0]
    
    print(i + 1., title_from_index )
    
    i +=  1
    
    if i > 9:
        break
    

The most recommended movies to  Fight Club are: 

1.0 Crazy in Alabama
2.0 Panic Room
3.0 Much Ado About Nothing
4.0 The Heart of Me
5.0 A Room with a View
6.0 The Young and Prodigious T.S. Spivet
7.0 Cinderella
8.0 Alice Through the Looking Glass
9.0 The 51st State
10.0 Howards End
